## 1. Import Airport data from aerodatabox(rapidAPI) for following airports and store it in CSV file
###Delhi (DEL), Bengaluru (BLR), Mumbai (BOM), Hyderabad (HYD), Chennai (MAA), Mangalore (IXE), Varanasi (VNS), ###hmedabad (AMD), Heathrow (LHR), Dubai (DXB), Abu Dhabi (AUH), Paris (CDG for Charles de Gaulle), New York (JFK ###for John F. Kennedy), LA (LAX for Los Angeles), and San Francisco (SFO). 


In [1]:
!pip3 install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import requests
import pandas as pd

airportsIATA=['DEL','BOM','BLR','MAA','HYD','IXE','VNS','AMD', 'LHR', 'JFK', 'DXB', 'AUH', 'CDG','LAX','SFO']
baseAPIUrl=r'https://aerodatabox.p.rapidapi.com/airports/iata/'

headers = {
	"x-rapidapi-key": "f6d7bb291fmsh9068a10ceb870dcp195adajsnc60927c769f5",
	"x-rapidapi-host": "aerodatabox.p.rapidapi.com"
}

airportsData=pd.DataFrame()

for iata in airportsIATA:
    url = baseAPIUrl + iata
    response = requests.get(url, headers=headers)
    df = pd.json_normalize(response.json())
    airportsData=pd.concat([airportsData,df], ignore_index=True)

airportsData.to_csv(r"C:\Guvi-Projects\Flight-Analytics\Data\airportsData.csv", index=False)
airportsData

## 2. Format data from csv file and Store it in local MySql database

In [ ]:
import mysql.connector
import pandas as pd
connection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "0123456789"
)
cursor = connection.cursor()

cursor.execute("CREATE DATABASE IF NOT EXISTS FlightDB")
cursor.execute("USE FlightDB")

airportsData=pd.read_csv(r"C:\Guvi-Projects\Flight-Analytics\Data\airportsData.csv")
reqCols = ['icao','iata','shortName','municipalityName','country.name','continent.name','location.lat','location.lon','timeZone']
data = list(airportsData[reqCols].itertuples(index=False,name=None ))

airportCreatequery = """CREATE TABLE IF NOT EXISTS Airports(
            icao VARCHAR(10), 
            iata VARCHAR(10) PRIMARY KEY, 
            name VARCHAR(100), 
            city VARCHAR(100), 
            country VARCHAR(100), 
            continent VARCHAR(50), 
            latitude FLOAT, 
            longitude FLOAT, 
            timeZone VARCHAR(50))"""
            
cursor.execute(airportCreatequery)

airportInsertQuery = r"INSERT INTO Airports (icao, iata, Name, city, country, continent, latitude, longitude, timeZone) VALUES ( %s, %s, %s, %s, %s, %s, %s, %s, %s )"
cursor.executemany(airportInsertQuery, data)
connection.commit()